### Creating the Database
* In order to create the database, please ensure `sqlalchemy_utils` and `psycopg2` has been installed in your local machine.

In [ ]:
# Import for creating a new database
# Note: make sure to install sqlalchemy_utils and psycopg2
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import create_engine
# Import classes to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, LargeBinary
# Import Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

* Please change `owner_username` and `password` accordingly, based on your database setup information.

In [ ]:
# Creating the database

owner_username = 'postgres'
password = 'jacky%402023'
host_name_address = 'localhost'

engine = create_engine(f"postgresql://{owner_username}:{password}@{host_name_address}/project4_db")

if not database_exists(engine.url):
    create_database(engine.url)

if database_exists(engine.url):
    print('Database was created successfully!')
else:
    print('Something went wrong.')

* Declare classes that will become the tables

In [ ]:
# Declare classes that will become the tables
Base = declarative_base()

# Create the Contacts class
class TrainImages(Base):
    __tablename__ = 'trainimages'
    image_id = Column(Integer, primary_key=True)
    file_name = Column(String)
    image_content = Column(LargeBinary)

class PriceTable(Base):
    __tablename__ = 'pricetable'
    id = Column(Integer, primary_key=True)
    Brand = Column(String)
    Category = Column(String)
    Color = Column(String)
    Size = Column(String)
    Material = Column(String)
    Price = Column(Float)

# Create tables
Base.metadata.create_all(engine)

### Extract Data From Raw Material Data Sources

* Extract data from `clothes_price_prediction_data.csv`

In [11]:
#  Import and read the clothes_price_prediction_data.csv
import pandas as pd 
df = pd.read_csv("clothes_price_prediction_data.csv")
print(f'There are {len(df)} row in the dataframe.')
print('='*50)
print(df.head())


There are 1000 row in the dataframe.
          Brand Category  Color Size Material  Price
0   New Balance    Dress  White   XS    Nylon    182
1   New Balance    Jeans  Black   XS     Silk     57
2  Under Armour    Dress    Red    M     Wool    127
3          Nike    Shoes  Green    M   Cotton     77
4        Adidas  Sweater  White    M    Nylon    113


* Extract data from images

In [ ]:
# import cv2
# import os

# def load_images_from_folder(folder):
#     images = []
#     for filename in os.listdir(folder):
#         img = cv2.imread(os.path.join(folder,filename))
#         if img is not None:
#             images.append(img)
#     return images

# OR ---------------------------------------------------------------
# from skimage.io import imread_collection

# #your path 
# col_dir = 'cats/*.jpg'

# #creating a collection with the available images
# col = imread_collection(col_dir)
# # You can create a collection and access elements the standard way, i.e. col[index]. This will give you the RGB values.

### Load Data to Database

In [ ]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import delete

* Load CSV Data to Database

In [ ]:
# Define function to add df to database table
# How to use method:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
def add_df_to_db(table_name, df):
    engine.execute(f"DELETE FROM {table_name}")

    rows_added = df.to_sql(table_name, engine, if_exists='append', index=False)

    print(f"{rows_added} rows were added successfully to {table_name} table.")


# Add contacts df to database
add_df_to_db('pricetable', df)

* Load Image Data to Database

In [ ]:
# Create a session maker
Session = sessionmaker(bind=engine)

# Create a session
session = Session()

# Clear the TrainImages table
session.execute(delete(TrainImages))

# Commit the deletion
session.commit()

# Loop over each image loaded from S3
for idx, img in enumerate(images):
    # Generate a unique file name
    file_name = f'image_{idx}.png'

    # Convert image to bytes
    with BytesIO() as output:
        plt.imsave(output, img, format='png')
        image_data = output.getvalue()

    # Create a new TrainImages instance and set its attributes
    image_instance = TrainImages(file_name=file_name, image_content=image_data)

    # Add the instance to the session
    session.add(image_instance)

# Commit the changes to the database
session.commit()

# Query all records from the TrainImages table and print file names
train_images = session.query(TrainImages).all()
for image in train_images:
    print(image.file_name)

# Close the session
session.close()